In [1]:
!------------------------------------------------------------------
!Creating module for constants
!------------------------------------------------------------------ 
module Constants
    integer :: N  !Number of bodies
    real(8), allocatable :: m(:)
    real(8) :: G = 39.478d0!Gravitational Constant in AU^3/(yr^2 M_sun) !Normalize system to G = 1
    real(8) :: pi = 3.14159265d0
    real(8) :: M1 = 0.5d0 !Mass of particles 
    real(8) :: Rc = 1000.d0
    real(8) :: MCore = 300.d0
    real(8) :: Mout = 100000.d0
    real(8) :: Epsilon = 10.d0
end module Constants

!------------------------------------------------------------------
! Creating subroutine for RHS vector
!------------------------------------------------------------------ 
subroutine RHS(Narray, ypos, yvel, t, fvel, facc)
    use Constants
    implicit none
    integer, intent(in) :: Narray !Number of components of position or velocity array 3*N
    real(8), dimension(Narray), intent(in) :: ypos, yvel
    real(8), dimension(Narray), intent(out) :: fvel, facc
    real(8), intent(in) :: t
    real(8) :: vcirc
    real(8), dimension(N)  :: phi
    real(8), dimension(Narray) :: Neg_Del_Phi
    real(8), dimension(3) :: ri, rj, rvec, acc
    real(8) :: sep
    integer :: i, j 
    
    fvel(1:Narray) = yvel(1:Narray)
    do i = 1, N
        ri(1:3) = ypos(3*i - 2: 3*i)
        acc = 0.d0
        do j = 1, N
            if(i == j) cycle 
            rj(1:3) = ypos(3*j - 2: 3*j)
            rvec = ri - rj
            sep = SQRT(dot_product(rvec, rvec))
            acc = acc - (m(j)*rvec)/(sep**2 + Epsilon**2)**1.5d0 
        end do
        acc(1) = acc(1) -(Mout)*((ri(1))/(ri(1)**2 + ri(2)**2 + ri(3)**2 + (2*Rc)**2)**(1.5d0))
        acc(2) = acc(2) -(Mout)*((ri(2))/(ri(1)**2 + ri(2)**2 + ri(3)**2 + (2*Rc)**2)**(1.5d0))
        acc(3) = acc(3) -(Mout)*((ri(3))/(ri(1)**2 + ri(2)**2 + ri(3)**2 + (2*Rc)**2)**(1.5d0))
        facc(3*i - 2: 3*i) = acc(1:3)
    end do
end subroutine RHS

!----------------------------------------------------------------------------
! Creating subroutine for fourth-order Runge-Kutta integrator
!----------------------------------------------------------------------------
subroutine RK4(Narray, ypos, yvel, t, delta_t)
    use Constants
    implicit none
    integer, intent(in) :: Narray !Number of components of position or velocity array
    real(8), intent(in) :: t, delta_t
    real(8), dimension(Narray), intent(inout) :: ypos, yvel
    real(8), dimension(Narray) :: ptmp1, ptmp2, ptmp3
    real(8), dimension(Narray) :: vtmp1, vtmp2, vtmp3
    real(8), dimension(Narray) :: f1v, f2v, f3v, f4v
    real(8), dimension(Narray) :: f1a, f2a, f3a, f4a
    real(8), dimension(Narray) :: fvel, facc
    real(8) :: h
    
    
    h = 0.5d0*delta_t

    call RHS(Narray, ypos, yvel, t, fvel, facc)
    f1v = delta_t*fvel
    f1a = delta_t*facc
    vtmp1 = yvel + 0.5d0*f1a
    ptmp1 = ypos + 0.5d0*f1v
    call RHS(Narray, ptmp1, vtmp1 , t + h, fvel, facc)
    f2v = delta_t*fvel
    f2a = delta_t*facc
    vtmp2 = yvel + 0.5d0*f2a
    ptmp2 = ypos + 0.5d0*f2v
    call RHS(Narray, ptmp2, vtmp2, t + h, fvel, facc)
    f3v = delta_t*fvel
    f3a = delta_t*facc
    vtmp3 = yvel + 0.5d0*f3a
    ptmp3 = ypos + 0.5d0*f3v
    call RHS(Narray, ptmp3, vtmp3, t + delta_t, fvel, facc)
    f4v = delta_t*fvel
    f4a = delta_t*facc
    
    yvel = yvel + (f1a + 2.d0*f2a + 2.d0*f3a + f4a)/6.d0
    ypos = ypos + (f1v + 2.d0*f2v + 2.d0*f3v + f4v)/6.d0
end subroutine RK4

!----------------------------------------------------------------------------
! Kinematic subroutines
!----------------------------------------------------------------------------
subroutine angular_momentum(Narray, ypos, yvel, Jspec, L_tot)
    use constants
    implicit none 
    integer, intent(in) :: Narray
    real(8), dimension(Narray), intent(in) :: ypos, yvel 
    real(8), dimension(Narray), intent(out) :: Jspec  
    real(8), dimension(3), intent(out) :: L_tot 
    real(8), dimension(3) :: ri, vi
    integer :: i

    L_tot(1:3) = 0.d0
    do i = 1, N        
        ri(1:3) = ypos(3*i - 2: 3*i)
        vi(1:3) = yvel(3*i - 2: 3*i) 
        Jspec(3*i - 2) = ri(2)*vi(3) - ri(3)*vi(2)  
        Jspec(3*i - 1) = -(ri(1)*vi(3) - ri(3)*vi(1)) 
        Jspec(3*i) = ri(1)*vi(2) - ri(2)*vi(1)
        L_tot = L_tot + m(i)*Jspec(3*i - 2:3*i)
    end do
end subroutine angular_momentum

subroutine Kinetic_Energy(Narray, yvel, E_kin, E_kin_tot)
    use constants
    implicit none
    integer, intent(in) :: Narray 
    real(8), dimension(Narray), intent(inout) :: yvel    
    real(8), dimension(N), intent(out) :: E_kin 
    real(8), intent(out) :: E_kin_tot 
    real(8), dimension(3) :: vel_vec
    real(8) :: vel_mag2
    integer :: i
    
    E_kin(1:N) = 0.d0
    E_kin_tot = 0.d0
    do i = 1, N
        vel_vec(1:3) = yvel(3*i -2: 3*i) 
        vel_mag2 = dot_product(vel_vec, vel_vec)
        E_kin(i) = 0.5d0*m(i)*vel_mag2
        E_kin_tot = E_kin_tot + 0.5d0*(m(i))*vel_mag2
    end do
end subroutine Kinetic_Energy

subroutine Potential_Energy(Narray, ypos, Phi, Neg_Del_Phi, E_pot, E_pot_tot)
    use constants
    implicit none
    integer, intent(in) :: Narray
    real(8), dimension(Narray), intent(inout) :: ypos 
    real(8), dimension(N), intent(out) :: E_pot 
    real(8), intent(out) :: E_pot_tot 
    real(8), dimension(N) :: Phi 
    real(8), dimension(Narray) :: Neg_Del_Phi
    real(8), dimension(3) :: ri, rj, dr
    real(8) :: seperation
    integer :: i, j
    
    E_pot(1:N) = 0.d0
    E_pot_tot = 0.d0
    do i = 1, N
        ri(1:3) = ypos(3*i - 2: 3*i) 
        E_pot(i) = 0.d0
        do j = i + 1, N
            if (i == j) cycle 
            rj(1:3) = ypos(3*j - 2: 3*j) 
            dr = ri - rj 
            seperation = ABS(SQRT(dot_product(dr, dr)))
            E_pot(i) = -m(i)*m(j)/(seperation**2 + Epsilon**2)**(0.5d0) 
        end do
        E_pot(i) = E_pot(i) + m(i)*(-Mout/(ri(1)**2 + ri(2)**2 + ri(3)**2 + (2*Rc)**2)**(0.5d0)) 
        E_pot_tot = E_pot_tot + E_pot(i)
    end do
end subroutine Potential_Energy

!----------------------------------------------------------------------------
! Timestep adjustor
!----------------------------------------------------------------------------
real(8) function timestep_adjuster(Narray, ypos, yvel, t, t_end, prec)
    use constants
    implicit none
    integer, intent(in) :: Narray
    real(8), dimension(Narray), intent(in) :: ypos, yvel
    real(8), intent(in) :: t, t_end, prec
    real(8), dimension(3) :: ri, rj, vi, vj 
    real(8), dimension(3) :: dr, dv
    real(8) :: dr_squared, dv_squared, R_mag, v_mag
    real(8) :: delta_t
    integer :: i, j
    
    delta_t = 10**50 !Just setting a huge number
    do i = 1, (N - 1)
        ri(1:3) = ypos(3*i - 2:3*i)
        vi(1:3) = yvel(3*i - 2:3*i)
        do j = i + 1, N
            if(i == j) cycle
            rj(1:3) = ypos(3*j - 2:3*j)
            vj(1:3) = yvel(3*j - 2:3*j)

            dr_squared = 0.d0
            dv_squared = 0.d0
            
            dr = ri - rj
            dr_squared = dot_product(dr, dr)
            R_mag = SQRT(dr_squared) 
            
            dv = vi - vj
            dv_squared = dot_product(dv, dv)
            v_mag = SQRT(dv_squared) 
            delta_t = min(delta_t, prec*(R_mag/v_mag))
        end do
    end do
    timestep_adjuster = min(delta_t, t_end - t)
end function timestep_adjuster

!----------------------------------------------------------------------------
! Subroutines for initial time and positions
!----------------------------------------------------------------------------
subroutine initial_positions(Narray, ypos)
    use constants
    implicit none
    integer, intent(in) :: Narray
    real(8), dimension(Narray), intent(inout) :: ypos
    real(8) :: x, y, z
    real(8) :: xi_ini_pos_x, xi_ini_pos_y,xi_ini_pos_z
    integer :: i
    
    do i = 1, N
        !Go back to 1 if less more than R^2 and resetting x, y, z
        1 x = 0.d0
        y = 0.d0
        z = 0.d0
        call random_number(xi_ini_pos_x)
        xi_ini_pos_x = 2*xi_ini_pos_x - 1
        call random_number(xi_ini_pos_y)
        xi_ini_pos_y = 2*xi_ini_pos_y - 1
        call random_number(xi_ini_pos_z)
        xi_ini_pos_z = 2*xi_ini_pos_z - 1
        if (xi_ini_pos_x**2 + xi_ini_pos_y**2 > 1.d0) then 
            goto 1
        end if
        if (xi_ini_pos_x**2 + xi_ini_pos_y**2 <= 1.d0) then
            x = Rc*xi_ini_pos_x + 50000 !!! Distance flag !!!
            ypos(3*i - 2) = x
            y = Rc*xi_ini_pos_y
            ypos(3*i - 1) = y
            z = Rc*xi_ini_pos_z
            ypos(3*i) = z
        end if
    end do
end subroutine initial_positions

subroutine initial_velocities(Narray, yvel)
    use constants
    implicit none
    integer, intent(in) :: Narray
    real(8), dimension(Narray), intent(inout) :: yvel
    real(8) :: v_rms, chi 
    real(8) :: xi_x, xi_y, xi_z, vx, vy, vz 
    real(8), dimension(N) :: chi_array, velvec_squared_array 
    real(8) :: A_norm_squared, B_norm
    integer :: i
    
    v_rms = sqrt(Mcore/Rc)
    do i = 1, N
        call random_number(chi)
        chi_array(i) = chi
    end do
    A_norm_squared = dot_product(chi_array, chi_array)/(N*(v_rms**2))
    do i = 1, N
        velvec_squared_array(i) = A_norm_squared*(chi_array(i)**2)
    end do
    do i = 1, N
        B_norm = 0.d0
        !Go back to 2 if outside of bounds and reset
        2 vx = 0.d0
        vy = 0.d0
        vz = 0.d0
        call random_number(xi_x)
        xi_x =  2*xi_x - 1
        call random_number(xi_y) 
        xi_y =  2*xi_y - 1 
        call random_number(xi_z)
        xi_z =  2*xi_z - 1
        if (xi_x**2 + xi_y**2 > 1.d0) then !If greater than 1^2
            goto 2
        end if
        if (xi_x**2 + xi_y**2 <= 1.d0) then !If less than or equal to 1^2
            B_norm = abs(sqrt(velvec_squared_array(i)/xi_x**2 + xi_y**2))
            vx = B_norm*xi_x*.001
            yvel(3*i - 2) = vx
            vy = B_norm*xi_y*.001
            yvel(3*i - 1) = vy
            vz = B_norm*xi_z*.001
            yvel(3*i) = vz
        end if
    end do
end subroutine initial_velocities

!----------------------------------------------------------------------------
! Creating subroutine that calculates the five closest neighbors to a particle
!----------------------------------------------------------------------------
subroutine neighbors(Narray, ypos, dist_list, idist_list) 
    use constants
    implicit none
    integer, intent(in) :: Narray
    real(8), dimension(Narray), intent(in) :: ypos
    real(8), dimension(N), intent(out) :: dist_list, idist_list
    real(8), dimension(5) :: dist_list_5, idist_list_5
    real(8), dimension(3) :: pos_i, pos_j
    real(8) :: dist_2
    integer :: i, j

    do i = 1, N
        dist_list_5(:) = huge(dist_2)
        idist_list_5 = 0.0 
        do j = 1, N
            if (i == j) cycle
            pos_i(1:3) = ypos(3*i - 2: 3*i) 
            pos_j(1:3) = ypos(3*j - 2: 3*j) 
            dist_2 = sqrt((pos_i(1) - pos_j(1))**2 + (pos_i(2) - pos_j(2))**2 + (pos_i(3) - pos_j(3))**2)
            if (dist_2 .gt. dist_list_5(5)) then 
                cycle
            else if ((dist_2 .le. dist_list_5(5)) .and. (dist_2 .gt. dist_list_5(4))) then
                dist_list_5(5) = dist_2
                idist_list_5(5) = j
            else if ((dist_2 .le. dist_list_5(4)) .and. (dist_2 .gt. dist_list_5(3))) then
                dist_list_5(5) = dist_list_5(4)
                idist_list_5(5) = idist_list_5(4) 
                dist_list_5(4) = dist_2
                idist_list_5(4) = j
            else if ((dist_2 .le. dist_list_5(3)) .and. (dist_2 .gt. dist_list_5(2))) then
                dist_list_5(5) = dist_list_5(4)
                idist_list_5(5) = idist_list_5(4)
                dist_list_5(4) = dist_list_5(3)
                idist_list_5(4) = idist_list_5(3)
                dist_list_5(3) = dist_2
                idist_list_5(3) = j
            else if ((dist_2 .le. dist_list_5(2)) .and. (dist_2 .gt. dist_list_5(1))) then
                dist_list_5(5) = dist_list_5(4)
                idist_list_5(5) = idist_list_5(4)
                dist_list_5(4) = dist_list_5(3)
                idist_list_5(4) = idist_list_5(3)
                dist_list_5(3) = dist_list_5(2)
                idist_list_5(3) = idist_list_5(2)
                dist_list_5(2) = dist_2
                idist_list_5(2) = j
            else
                dist_list_5(5) = dist_list_5(4)
                idist_list_5(5) = idist_list_5(4)
                dist_list_5(4) = dist_list_5(3)
                idist_list_5(4) = idist_list_5(3)
                dist_list_5(3) = dist_list_5(2)
                idist_list_5(3) = idist_list_5(2)
                dist_list_5(2) = dist_list_5(1)
                idist_list_5(2) = idist_list_5(1)
                dist_list_5(1) = dist_2
                idist_list_5(1) = j
            end if
        end do
        idist_list(i) = idist_list_5(5)
        dist_list(i) = dist_list_5(5)
    end do
end subroutine neighbors

!----------------------------------------------------------------------------
! Creating subroutine that calculates if simulation should be cut off
!----------------------------------------------------------------------------
subroutine stopper(Narray, ypos, yes_or_no) 
    use constants
    implicit none
    integer, intent(in) :: Narray
    real(8), dimension(Narray), intent(in) :: ypos
    integer, intent(out) :: yes_or_no
    real(8), dimension(3) :: pos
    real(8) :: half_RC
    integer :: i, in_half_RC
    
    half_RC = Rc/2
    in_half_RC = 0
    
    do i = 1, N
        pos(1:3) = ypos(3*i - 2: 3*i)
        if ((pos(1) .le. half_RC) .and. (pos(2) .le. half_RC) .and. (pos(3) .le. half_RC)) then
            in_half_RC = in_half_RC + 1
        end if
    end do
    if (in_half_RC .ge. (N/10)) then
        yes_or_no = 1
    else
        yes_or_no = 0
    end if 
end subroutine stopper

!----------------------------------------------------------------------------
! Begin main program
!----------------------------------------------------------------------------
program orbit_simulator
    use constants
    implicit none
    real(8), allocatable :: ypos(:), yvel(:), fvel(:), facc(:)
    real(8), allocatable :: E_kin(:), E_pot(:), Jspec(:)
    real(8), allocatable :: dist_list(:), idist_list(:)
    real(8), allocatable :: phi(:), Neg_Del_Phi(:)
    real(8) :: E_kin_tot, E_pot_tot
    real(8), dimension(3) :: L_tot
    real(8), dimension(3) :: L_tot_ini, L_tot_fin
    real(8) :: E_tot_ini, E_tot_fin, E_error, L_error
    real(8) :: M_tot
    real(8) :: t, t_end, delta_t
    real(8) :: avg_distance
    real(8) :: r_2, v_2circ, v_rms 
    real(8), external :: timestep_adjuster
    real(8), parameter :: prec = 1.E-4
    integer :: Narray, Nsteps, i
    integer :: yes_or_no
    
    !!! Choose number of bodies here and allocate!!!
    N = 50
    Narray = 3*N
    
    allocate(m(N),dist_list(N),idist_list(N))
    allocate(E_kin(N), E_pot(N),Phi(N))
    allocate(ypos(Narray),yvel(Narray), fvel(Narray), facc(Narray))
    allocate(Jspec(Narray), Neg_Del_Phi(Narray))
    !------------------------------------------------------------------
    ! Initialize system
    !------------------------------------------------------------------    
    !!! Initial Conditions !!!
    do i = 1, N
        m(i) = M1
    end do
    call initial_positions(Narray, ypos)
    call initial_velocities(Narray, yvel)
    
    M_tot = m(1)*N 
    v_rms = sqrt(Mcore/Rc)
    
    t = 0.d0
    t_end = N*((2*Rc)/v_rms)*10
    Nsteps = 0
    
    call angular_momentum(Narray, ypos, yvel, Jspec, L_tot)
    call Kinetic_Energy(Narray, yvel, E_kin, E_kin_tot)
    call Potential_Energy(Narray, ypos, Phi, Neg_Del_Phi, E_pot, E_pot_tot)
    call neighbors(Narray, ypos, dist_list, idist_list)
    
    avg_distance = sum(dist_list)/N
         
    L_tot_ini = L_tot
    E_tot_ini = E_kin_tot + E_pot_tot
    !------------------------------------------------------------------
    ! Begin the simulation and record the results
    !------------------------------------------------------------------ 
    open(1, file = 'particle_data_3_new')
    do i = 1, N 
        write(1, *) t, ypos(3*i-2), ypos(3*i-1), ypos(3*i), i, idist_list(i), dist_list(i), avg_distance
    end do
    !Integration loop
    do while(t < t_end)
        delta_t = timestep_adjuster(Narray, ypos, yvel, t, t_end, prec)
        call RK4(Narray, ypos, yvel, t, delta_t)
        call angular_momentum(Narray, ypos, yvel, Jspec, L_tot)
        call neighbors(Narray, ypos, dist_list, idist_list)
        
        avg_distance = sum(dist_list)/N

        Nsteps = Nsteps + 1
        t = t + delta_t
        
        !!! Stop condition ("yes") is a 1 !!!
        call stopper(Narray, ypos, yes_or_no) 
        if ((yes_or_no == 1)) then
            do i = 1, N
                write(1, *) t, ypos(3*i-2), ypos(3*i-1), ypos(3*i), i, idist_list(i), dist_list(i), avg_distance
            end do
            exit
        end if
        
        !!! Replace 10 with desired reduction in recorded data points !!!
        if (mod(Nsteps, 100) == 0) then 
            do i = 1, N
                write(1, *) t, ypos(3*i-2), ypos(3*i-1), ypos(3*i), i, idist_list(i), dist_list(i), avg_distance
            end do
        end if
    end do
    close(1)
    !------------------------------------------------------------------
    ! Check simulation validity/errors
    !------------------------------------------------------------------ 
    call angular_momentum(Narray, ypos, yvel, Jspec, L_tot)
    call Kinetic_Energy(Narray, yvel, E_kin, E_kin_tot)
    call Potential_Energy(Narray, ypos, Phi, Neg_Del_Phi, E_pot, E_pot_tot)
     
    E_tot_fin = E_kin_tot + E_pot_tot
    L_tot_fin = L_tot 
    E_error = abs(1.d0 - (E_kin_tot + E_pot_tot)/E_tot_ini)
    L_error = abs(1.d0 - L_tot_fin(3)/L_tot_ini(3))
    
    print *, "Simulation Information:"
    print *, "Time Elapsed:", t
    print *, "Number of Steps:", Nsteps
    print *, "Conservation of Energy - Error:", E_error
    print *, "Conservation of Angular Momentum - Error:", L_error
    
end program orbit_simulator

 Simulation Information:
 Time Elapsed:   1.3536773550890183E-003
 Number of Steps:           1
 Conservation of Energy - Error:   2.1678991934948044E-011
 Conservation of Angular Momentum - Error:   9.8809849191638932E-013
